In [1]:
import pandas as pd

Census bureau definition of fips_class_codes:

    fips_class_codes
    
- H1:  identifies an active county or statistically equivalent entity that does not qualify under subclass C7 or H6.
- H4:  identifies a legally defined inactive or nonfunctioning county or statistically equivalent entity that does not qualify under subclass H6.
- H5:  identifies census areas in Alaska, a statistical county equivalent entity.
- H6:  identifies a county or statistically equivalent entity that is areally coextensive or governmentally consolidated with an incorporated place, part of an incorporated place, or a consolidated city. 
- C7:  identifies an incorporated place that is an independent city; that is, it also serves as a county equivalent because it is not part of any county, and a minor civil division (MCD) equivalent because it is not part of any MCD.

Wikipedia says:
    
    As of 2013, the United States has 3,007 counties and 137 county
    equivalents for a total of 3,144 counties and county equivalents.

## Flow

1. [Import US county level FIPS codes and names from the Census Dept](#Import-Census-data)
2. [Import Guardian newspaper 2012 county-level election results](#Import-Guardian-data)
3. [Parse and clean Guardian data, particularly their interpretation of FIPS codes and county definitions](#Clean-Guardian-data)
4. [Combine Census FIPS codes and county names with Guardian county-level election results](#Combine-datasets)
5. [Export resulting dataset](#Export-results)

## Import Census data

In [2]:
# county_fips data from https://www.census.gov/geo/reference/codes/cou.html
census = pd.read_csv('http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',sep=',',header=None, dtype=str)
census.columns = ['state_abbr', 'state_fips', 'county_fips', 'county_name', 'fips_class_code']
print(census.shape)
census.head()

(3235, 5)


,state_abbr,state_fips,county_fips,county_name,fips_class_code
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [3]:
# look at some states data
census[census['state_abbr'] == 'DC']

,state_abbr,state_fips,county_fips,county_name,fips_class_code
319,DC,11,001,District of Columbia,H6


In [4]:
# group by distinct fips class codes and return the count
fips_census_group = census.groupby(by='fips_class_code',axis=0)['state_abbr'].count()
fips_census_group

fips_class_code
C7      42
H1    3126
H4      30
H5      11
H6      26
Name: state_abbr, dtype: int64

In [5]:
# Look at independent cities
# should include Baltimore, Maryland; Carson City, Nevada; St. Louis, Missouri; and all 38 cities in Virginia
ind_census_cities = census[census['fips_class_code'] == 'C7']
print(ind_census_cities['state_abbr'].count())
ind_census_cities.head()

42


,state_abbr,state_fips,county_fips,county_name,fips_class_code
1216,MD,24,510,Baltimore city,C7
1597,MO,29,510,St. Louis city,C7
1763,NV,32,510,Carson City,C7
2915,VA,51,510,Alexandria city,C7
2916,VA,51,515,Bedford city,C7


In [6]:
# look at Alaska county equivalents
alaska_census_counties = census[census['fips_class_code'] == 'H5']
print(alaska_census_counties['state_abbr'].count())
alaska_census_counties

11


,state_abbr,state_fips,county_fips,county_name,fips_class_code
68,AK,02,016,Aleutians West Census Area,H5
70,AK,02,050,Bethel Census Area,H5
73,AK,02,070,Dillingham Census Area,H5
76,AK,02,105,Hoonah-Angoon Census Area,H5
83,AK,02,180,Nome Census Area,H5
86,AK,02,195,Petersburg Census Area,H5
87,AK,02,198,Prince of Wales-Hyder Census Area,H5
90,AK,02,240,Southeast Fairbanks Census Area,H5
91,AK,02,261,Valdez-Cordova Census Area,H5
92,AK,02,270,Wade Hampton Census Area,H5


In [7]:
# get state and county fips
#census.columns[[0,1,2,3,4]]
fips_codes_census = census.drop(census.columns[[4]],axis=1)
print(fips_codes_census['county_fips'].count())
fips_codes_census.head()

3235


,state_abbr,state_fips,county_fips,county_name
0,AL,01,001,Autauga County
1,AL,01,003,Baldwin County
2,AL,01,005,Barbour County
3,AL,01,007,Bibb County
4,AL,01,009,Blount County


## Import Guardian data

In [8]:
# import Guardian data
# https://www.theguardian.com/news/datablog/2012/nov/07/us-2012-election-county-results-download#data
# NOTE: xlrd package needed (not installed by deafault) 
# to read data and format information from older Excel files (ie: .xls)
# get all data within the 'FULL DATA' worksheet and pass it into a DataFrame
guardian_full = pd.read_excel('http://image.guardian.co.uk/sys-files/Guardian/documents/2012/11/14/US_elect_county.xls',
                     sheetname = 'FULL DATA',
                     header = 0,
                     index_col=None,
                     convert_float = True,
                     converters={'FIPS Code':str},
                     parse_cols = "A, D, E, K, M, T, Y, AF, AK, AR, AW, BD, BI, BP")
                     
guardian_full.head()

WARNING *** file size (3825153) not 512 + multiple of sector size (512)


,State Postal,FIPS Code,County Name,TOTAL VOTES CAST,Party,Votes,Party.1,Votes.1,Party.2,Votes.2,Party.3,Votes.3,Party.4,Votes.4
0,AK,0,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN
1,AK,2000,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN
2,AL,0,Alabama,2064699,Dem,793620,GOP,1252453,Ind,2961.0,Ind,12283.0,Ind,3382.0
3,AL,1001,Autauga,23909,Dem,6354,GOP,17366,Ind,31.0,Ind,136.0,Ind,22.0
4,AL,1003,Baldwin,84988,Dem,18329,GOP,65772,Ind,122.0,Ind,597.0,Ind,168.0


## Clean Guardian data

In [9]:
# return results for only Democrats and Republicans
def dem_func(x):
    if x['Party'] == 'Dem':
        return x['Votes']
    elif x['Party.1'] == 'Dem':
        return x['Votes.1']
    elif x['Party.2'] == 'Dem':
        return x['Votes.2']
    elif x['Party.3'] == 'Dem':
        return x['Votes.3']
    elif x['Party.4'] == 'Dem':
        return x['Votes.4']

def gop_func(x):
    if x['Party'] == 'GOP':
        return x['Votes']
    elif x['Party.1'] == 'GOP':
        return x['Votes.1']
    elif x['Party.2'] == 'GOP':
        return x['Votes.2']
    elif x['Party.3'] == 'GOP':
        return x['Votes.3']
    elif x['Party.4'] == 'GOP':
        return x['Votes.4']
    
guardian = guardian_full
guardian['votes_Dem'] = guardian_full.apply(lambda x: dem_func(x), axis=1)
guardian['votes_GOP'] = guardian_full.apply(lambda x: gop_func(x), axis=1)
guardian.head()

,State Postal,FIPS Code,County Name,TOTAL VOTES CAST,Party,Votes,Party.1,Votes.1,Party.2,Votes.2,Party.3,Votes.3,Party.4,Votes.4,votes_Dem,votes_GOP
0,AK,0,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN,91696.0,121234.0
1,AK,2000,Alaska,220596,Dem,91696,GOP,121234,Lib,5539.0,Grn,2127.0,NaN,NaN,91696.0,121234.0
2,AL,0,Alabama,2064699,Dem,793620,GOP,1252453,Ind,2961.0,Ind,12283.0,Ind,3382.0,793620.0,1252453.0
3,AL,1001,Autauga,23909,Dem,6354,GOP,17366,Ind,31.0,Ind,136.0,Ind,22.0,6354.0,17366.0
4,AL,1003,Baldwin,84988,Dem,18329,GOP,65772,Ind,122.0,Ind,597.0,Ind,168.0,18329.0,65772.0


In [10]:
# drop irrelevant columns
guardian.drop(labels=['Votes','Votes.1','Votes.2','Votes.3','Votes.4','Party','Party.1','Party.2','Party.3','Party.4'],axis=1,inplace=True)
# format dataframe headers
guardian.columns = ['state_abbr','combined_fips', 'county_name','votes_total','votes_Dem','votes_Repub']
# view votes_total cast results by state, county_fips, county_name, and candidate
print('FIPS code data type is:',guardian['combined_fips'].dtype)
guardian.head()

FIPS code data type is: object


,state_abbr,combined_fips,county_name,votes_total,votes_Dem,votes_Repub
0,AK,0,Alaska,220596,91696.0,121234.0
1,AK,2000,Alaska,220596,91696.0,121234.0
2,AL,0,Alabama,2064699,793620.0,1252453.0
3,AL,1001,Autauga,23909,6354.0,17366.0
4,AL,1003,Baldwin,84988,18329.0,65772.0


Guardian data summarizes county-level data to the state level within the same table. We need to create a dataframe that segregate county- and state-level data

In [11]:
# guardian data combines state- and county-level fips codes into one code
# extract correct county fips codes
guardian['county_fips'] = guardian['combined_fips'].str.extract('(.{3})\s*$',expand=False)
# extract correct state fips codes
guardian['state_fips_a'] = guardian['combined_fips'].apply(lambda x: x[0:len(x) - 3])

state_fips = []
for i,r in guardian['combined_fips'].iteritems():
    if len(r) > 4:
        state_fips.append(r[0:2])
    else:
        state_fips.append('0' + r[0:1])

guardian['state_fips'] = pd.Series(state_fips)

# filter out state-level rows
guardian = guardian[guardian['state_fips'] != '00']

guardian.head()

,state_abbr,combined_fips,county_name,votes_total,votes_Dem,votes_Repub,county_fips,state_fips_a,state_fips
1,AK,2000,Alaska,220596,91696.0,121234.0,000,2,02
3,AL,1001,Autauga,23909,6354.0,17366.0,001,1,01
4,AL,1003,Baldwin,84988,18329.0,65772.0,003,1,01
5,AL,1005,Barbour,11459,5873.0,5539.0,005,1,01
6,AL,1007,Bibb,8391,2200.0,6131.0,007,1,01


## Combine datasets

In [12]:
# join census county names on count and state FIPS
combined = pd.merge(census, guardian, on=['state_fips','county_fips'])
print(combined.shape[0])
combined.head()

4587


,state_abbr_x,state_fips,county_fips,county_name_x,fips_class_code,state_abbr_y,combined_fips,county_name_y,votes_total,votes_Dem,votes_Repub,state_fips_a
0,AL,01,001,Autauga County,H1,AL,1001,Autauga,23909,6354.0,17366.0,1
1,AL,01,003,Baldwin County,H1,AL,1003,Baldwin,84988,18329.0,65772.0,1
2,AL,01,005,Barbour County,H1,AL,1005,Barbour,11459,5873.0,5539.0,1
3,AL,01,007,Bibb County,H1,AL,1007,Bibb,8391,2200.0,6131.0,1
4,AL,01,009,Blount County,H1,AL,1009,Blount,23980,2961.0,20741.0,1


In [13]:
# drop irrelevant columns
county_level_combined = combined.drop(combined.columns[[0,1,4,6,7]], axis=1)

# recombine fips data
county_level_combined['combined_fips'] = county_level_combined['state_fips_a'] + county_level_combined['county_fips']

# drop irrelevant columns
county_level_combined.drop(county_level_combined.columns[[0,6]], axis=1,inplace=True)

#reformat column labels
county_level_combined.columns = ['county_name','state_abbr','votes_total','votes_dem','votes_gop','combined_fips']

county_level_combined.head()

,county_name,state_abbr,votes_total,votes_dem,votes_gop,combined_fips
0,Autauga County,AL,23909,6354.0,17366.0,1001
1,Baldwin County,AL,84988,18329.0,65772.0,1003
2,Barbour County,AL,11459,5873.0,5539.0,1005
3,Bibb County,AL,8391,2200.0,6131.0,1007
4,Blount County,AL,23980,2961.0,20741.0,1009


In [14]:
# calculate percentage of total vote per major candidates
county_level_combined['per_dem'] = county_level_combined['votes_dem'] / county_level_combined['votes_total']
county_level_combined['per_gop'] = county_level_combined['votes_gop'] / county_level_combined['votes_total']
county_level_combined

,county_name,state_abbr,votes_total,votes_dem,votes_gop,combined_fips,per_dem,per_gop
0,Autauga County,AL,23909,6354.0,17366.0,1001,0.265758,0.726337
1,Baldwin County,AL,84988,18329.0,65772.0,1003,0.215666,0.773897
2,Barbour County,AL,11459,5873.0,5539.0,1005,0.512523,0.483376
3,Bibb County,AL,8391,2200.0,6131.0,1007,0.262186,0.730664
4,Blount County,AL,23980,2961.0,20741.0,1009,0.123478,0.864929
5,Bullock County,AL,5318,4058.0,1250.0,1011,0.763069,0.235051
6,Butler County,AL,9483,4367.0,5081.0,1013,0.460508,0.535801
7,Calhoun County,AL,46240,15500.0,30272.0,1015,0.335208,0.654671
8,Chambers County,AL,14562,6853.0,7596.0,1017,0.470608,0.521632
9,Cherokee County,AL,9761,2126.0,7494.0,1019,0.217806,0.767749


## Export results

In [15]:
county_level_combined.to_csv('US presidential election results by county.csv',sep=',')

In [16]:
# view resulting data by state
county_level_combined[county_level_combined['state_abbr'] =='HI']
# census[census['state_abbr']=='CO']

,county_name,state_abbr,votes_total,votes_dem,votes_gop,combined_fips,per_dem,per_gop
678,Hawaii County,HI,62653,46670.0,14541.0,15001,0.744896,0.232088
679,Honolulu County,HI,292925,201803.0,87255.0,15003,0.688924,0.297875
680,Kauai County,HI,25369,18638.0,6121.0,15007,0.734676,0.241279
681,Maui County,HI,48547,35979.0,11577.0,15009,0.741117,0.238470
